# KELOMPOK THREEASURE
Anggota Kelompok :
1. Steffany Claussia Fernanda - 24083010026
2. Fanny Widya Cahyani - 24083010045
3. Izzati Kamila Putri - 24083010059

# DESKRIPSI DATASET

Dataset yang kami gunakan adalah dataset hasil survei mahasiswa UPN “Veteran” Jawa Timur yang sebelumnya telah melalui proses **pembersihan (data cleaning) dan transformasi** sehingga lebih rapi dan siap dianalisis. Dataset ini terdiri dari **153 entri dengan 20 kolom** yang memuat data terkait fenomena **FOMO (Fear of Missing Out)**, kondisi finansial, serta dampaknya terhadap **psikologis dan perkuliahan**.

Dari keseluruhan kolom, terdapat **9 kolom numerik** dan **12 kolom kategorikal atau teks**. Secara umum, dataset ini memberikan gambaran yang cukup komprehensif mengenai hubungan antara kondisi finansial, perilaku FOMO, serta dampak psikologis pada mahasiswa, termasuk upaya yang mereka lakukan untuk menghadapi stres tersebut.

# Data Preprocessing

In [200]:
import pandas as pd
import numpy as np
import os
from pathlib import Path
import plotly.express as px
import plotly.graph_objects as go
from scipy.stats import pearsonr
from IPython.display import display

Import Library yang diperlukan untuk melakukan analisis.

In [201]:
df = pd.read_csv('Data Eda Threeasure_Cleaned.csv')
df.head()

,Timestamp,ketersediaan_responden,nama_lengkap,NPM,fakultas,program_studi,no_whatshapp,sering_merasa_FOMO,rata-rata_uang_saku_perbulan,pengeluaran_untuk_FOMO_per_bulan,kemampuan_mengelola_keuangan,frekuensi_fomo_pengeluaran,pengaruh_fomo_terhadap_emosi,frekuensi_stres_karena_finansial,frekuensi_hilang_semangat_kuliah_karena_tekanan_finansial,frekuensi_stres_fomo,kebutuhan_akan_dukungan_emosional_dan_bantuan_psikologis,frekuensi_kegiatan_karena_FOMO,adanya_strategi_untuk_mengatasi_stress,email_address
0,9/10/2025 12:35:31,Ya,A'shadieeyah Maryam,24051010040,Fakultas Arsitektur dan Desain,Arsitektur,82321706636,Ya,"Rp2,000,000","Rp500,000",2,3,3,3,3,3,Ya,3,Ya,24051010040@student.upnjatim.ac.id
1,9/10/2025 16:18:36,Ya,Alisyah Shafa Frida Ramadhani,24051010014,Fakultas Arsitektur dan Desain,Arsitektur,85707445579,Tidak,"Rp1,000,000","Rp30,000",3,1,1,1,1,1,Tidak,3,Tidak,24051010014@student.upnjatim.ac.id
2,9/12/2025 19:16:37,Ya,Auwallina,23013010046,Fakultas Ekonomi dan Bisnis,Akuntansi,85733449061,Ya,"Rp1,350,000","Rp200,000",5,4,1,1,1,1,Ya,15,Ya,23013010046@student.upnjatim.ac.id
3,9/11/2025 12:13:33,Ya,Chintya Azzahra Damayanti,24013010202,Fakultas Ekonomi dan Bisnis,Akuntansi,81330260712,Tidak,"Rp500,000","Rp150,000",3,2,2,2,2,1,Tidak,15,Tidak,24013010202@student.upnjatim.ac.id
4,9/10/2025 21:26:35,Ya,Devi Ayunda Nur Aini,24013010119,Fakultas Ekonomi dan Bisnis,Akuntansi,81230845767,Tidak,"Rp700,000","Rp300,000",4,2,1,1,1,1,Tidak,6,Ya,24013010119@student.upnjatim.ac.id


In [202]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 153 entries, 0 to 152
Data columns (total 20 columns):
 #   Column                                                     Non-Null Count  Dtype 
---  ------                                                     --------------  ----- 
 0   Timestamp                                                  153 non-null    object
 1   ketersediaan_responden                                     153 non-null    object
 2   nama_lengkap                                               153 non-null    object
 3   NPM                                                        153 non-null    int64 
 4   fakultas                                                   153 non-null    object
 5   program_studi                                              153 non-null    object
 6   no_whatshapp                                               153 non-null    int64 
 7   sering_merasa_FOMO                                         153 non-null    object
 8   rata-rata_uang_saku_

In [203]:
# Membuatskor kesejahteraan psikologis (semakin tinggi = semakin buruk)
df["skor_psikologis"] = (
    df["pengaruh_fomo_terhadap_emosi"] +
    df["frekuensi_stres_karena_finansial"] +
    df["frekuensi_hilang_semangat_kuliah_karena_tekanan_finansial"] +
    df["frekuensi_stres_fomo"]
)

In [204]:
# Warna
SALMON = "#FA8072"
SKYBLUE = "#87CEEB"
SALMON_GRAD = ["#C94A44", "#E96A5F", "#FF8C75", "#FFB5A7", "#FFE5E0"]

# Penyajian Data

## Jumlah Responden Harian: Analisis Time Series

In [205]:
df["Timestamp"] = pd.to_datetime(df["Timestamp"])

# Hitung jumlah responden per hari
responden_per_hari = df["Timestamp"].dt.date.value_counts().sort_index().reset_index()
responden_per_hari.columns = ["Tanggal", "Jumlah Responden"]

# Diagram garis tren jumlah responden
fig_ts = px.line(
    responden_per_hari,
    x="Tanggal",
    y="Jumlah Responden",
    markers=True,
    title="Tren Jumlah Responden yang Mengisi Survei per Hari",
    line_shape="spline",
    color_discrete_sequence=["SALMON"]
)
fig_ts.show()

Grafik menunjukkan tren jumlah responden yang mengisi survei per hari. Terlihat bahwa jumlah responden meningkat dari tanggal 9 hingga mencapai puncaknya pada tanggal 11 September 2025, kemudian mengalami penurunan yang cukup signifikan pada hari-hari berikutnya. Setelah penurunan tajam tersebut, jumlah responden relatif stabil dengan angka yang rendah hingga tanggal 17 September 2025. Pola ini menggambarkan antusiasme awal yang tinggi pada awal pengisian survei, lalu menurun seiring berjalannya waktu.

## Distribusi Responden Berdasarkan Fakultas

In [206]:
# Hitung jumlah responden per fakultas
fakultas_counts = df['fakultas'].value_counts().reset_index()
fakultas_counts.columns = ['Fakultas', 'Jumlah Responden']

# Membuatdiagram batang
fig = px.bar(
    fakultas_counts,
    x='Jumlah Responden',
    y='Fakultas',
    orientation='h',
    title='Distribusi Responden per Fakultas',
    color_discrete_sequence=[SKYBLUE],
)

fig.update_traces(texttemplate='%{x}', textposition='outside')

fig.update_layout(
    yaxis=dict(autorange="reversed"),
    xaxis_title='Jumlah Responden',
    yaxis_title='Fakultas',
    showlegend=False
)

fig.show()

Visualisasi diagram batang horizontal di atas memperlihatkan distribusi jumlah responden berdasarkan fakultas. Hasilnya menunjukkan bahwa mayoritas responden berasal dari **Fakultas Ilmu Komputer** dengan jumlah 54 orang, menjadikannya kelompok terbesar dalam penelitian ini. Posisi berikutnya ditempati oleh Fakultas Ekonomi dan Bisnis sebanyak 30 responden, serta Fakultas Teknik dan Sains dan Fakultas Ilmu Sosial Budaya dan Politik yang masing-masing berjumlah 25 dan 24 responden.

Sementara itu, fakultas dengan jumlah responden paling sedikit berasal dari Fakultas Arsitektur dan Desain dengan hanya 2 orang, disusul oleh Fakultas Hukum dan Fakultas Kedokteran yang masing-masing berjumlah 3 responden.


## Distribusi FOMO

In [207]:
# Jumlah responden berdasarkan jawaban sering_merasa_FOMO
fomo_counts = df['sering_merasa_FOMO'].value_counts().reset_index()
fomo_counts.columns = ['Sering Merasa FOMO', 'Jumlah']

# Membuatdiagram lingkaran dengan Plotly Express
fig_fomo = px.pie(
    fomo_counts,
    values='Jumlah',
    names='Sering Merasa FOMO',
    title='Proporsi Responden yang Sering Merasa FOMO',
    color_discrete_sequence=['SKYBLUE', 'SALMON']
)

fig_fomo.update_traces(textposition='inside', textinfo='percent+label', rotation=90)
fig_fomo.show()

Pada Pie Chart tersebut menunjukkan proporsi responden yang sering merasa FOMO. Sebanyak 36,6% responden menjawab “Ya” yang artinya 36.6% responden merasa FOMO, sedangkan 63,4% responden menjawab “Tidak”. Hasil ini memperlihatkan bahwa meskipun sebagian besar responden tidak terlalu dipengaruhi oleh FOMO, masih ada sekelompok signifikan yang cukup sering merasakannya.

In [208]:
# Hitung jumlah responden per fakultas berdasarkan FOMO
fomo_faculty = (
    df.groupby(["fakultas", "sering_merasa_FOMO"])
    .size()
    .reset_index(name="Jumlah")
)

# Membuat diagram batang
fig = px.bar(
    fomo_faculty,
    x="Jumlah",
    y="fakultas",
    color="sering_merasa_FOMO",
    barmode="group",
    orientation="h",
    title="Distribusi FOMO vs Tidak FOMO per Fakultas",
    color_discrete_sequence=["SKYBLUE", "SALMON"]
)

fig.update_layout(
    xaxis_title="Jumlah Responden",
    yaxis_title="Fakultas",
    plot_bgcolor="white",
    yaxis=dict(categoryorder="total ascending")
)

fig.show()


Diagram batang tersebut membandingkan distribusi responden yang sering merasa FOMO dan yang tidak pada masing-masing fakultas. Terlihat bahwa Fakultas Ilmu Komputer memiliki jumlah responden terbanyak baik pada kategori FOMO maupun tidak, dengan dominasi responden yang tidak sering merasa FOMO. Fakultas Ekonomi dan Bisnis serta Fakultas Teknik dan Sains juga memiliki jumlah responden cukup besar, dengan perbandingan antara FOMO dan tidak yang relatif seimbang. Sementara itu, fakultas lain seperti Pertanian, Kedokteran, dan Hukum menunjukkan jumlah responden yang lebih sedikit dengan kecenderungan variasi yang tidak terlalu signifikan. Hasil ini menunjukkan bahwa persebaran pengalaman FOMO bervariasi antar fakultas, namun sebagian besar tetap didominasi oleh kelompok yang tidak sering merasakannya.

## Distribusi Kemampuan Mengelola Keuangan

In [209]:
# Mapping kategori kemampuan mengelola keuangan
mapping_keuangan = {
    1: "Buruk",
    2: "Kurang Baik",
    3: "Cukup Baik",
    4: "Baik",
    5: "Sangat Baik"
}

df["kategori_keuangan"] = df["kemampuan_mengelola_keuangan"].map(mapping_keuangan)

# Hitung jumlah responden per kategori
keuangan_counts = df["kategori_keuangan"].value_counts().reset_index()
keuangan_counts.columns = ["Kategori", "Jumlah"]

# Urutkan sesuai tingkatan
order_keuangan = ["Sangat Baik", "Baik", "Cukup Baik", "Kurang Baik", "Buruk"]
keuangan_counts["Kategori"] = pd.Categorical(keuangan_counts["Kategori"], categories=order_keuangan, ordered=True)
keuangan_counts = keuangan_counts.sort_values("Kategori")

# Membuat diagram batang
fig = px.bar(
    keuangan_counts,
    x="Kategori",
    y="Jumlah",
    color="Kategori",
    color_discrete_sequence=SALMON_GRAD,
    title="Distribusi Kemampuan Mengelola Keuangan"
)

fig.update_layout(
    xaxis_title="Kategori Kemampuan",
    yaxis_title="Jumlah Responden",
    plot_bgcolor="white",
    xaxis=dict(showgrid=True, gridcolor="LightGrey"),
    yaxis=dict(showgrid=True, gridcolor="LightGrey")
)

fig.show()

Diagram batang di atas menunjukkan distribusi kemampuan responden dalam mengelola keuangan. Sebagian besar responden berada pada kategori cukup baik dan baik, dengan jumlah tertinggi pada kategori cukup baik. Responden dengan kemampuan sangat baik menempati posisi selanjutnya, sedangkan kategori kurang baik dan buruk hanya diisi oleh sedikit responden. Hasil ini mengindikasikan bahwa mayoritas responden memiliki kemampuan pengelolaan keuangan yang tergolong positif, meskipun masih ada sebagian kecil yang perlu meningkatkan keterampilan dalam hal tersebut.


## Distribusi Skor Psikologis Mahasiswa

In [210]:
# Membuat kategorisasi psikologis
def kategori_psikologis(skor):
    if skor <= 8:
        return "Baik"
    elif skor <= 15:
        return "Cukup Baik"
    else:
        return "Buruk"

df["kategori_psikologis_skor"] = df["skor_psikologis"].apply(kategori_psikologis)

# Hitung jumlah tiap kategori
kategori_counts_skor = df["kategori_psikologis_skor"].value_counts().reset_index()
kategori_counts_skor.columns = ["Kategori", "Jumlah"]

Kode tersebut digunakan untuk mengelompokkan skor psikologis responden ke dalam tiga kategori, yaitu baik, cukup baik, dan buruk. Skor dengan nilai delapan atau kurang dimasukkan ke dalam kategori baik, skor antara sembilan hingga lima belas masuk kategori cukup baik, sedangkan skor di atas lima belas dimasukkan ke dalam kategori buruk. Hasil kategorisasi ini kemudian disimpan dalam kolom baru pada dataset, dan jumlah responden untuk masing-masing kategori dihitung agar dapat dianalisis lebih lanjut, misalnya dengan membuat visualisasi distribusi kondisi psikologis responden.


In [211]:
# Boxplot distribusi skor psikologis
fig_box = px.box(
    df,
    y="skor_psikologis",
    points="all",
    title="Distribusi Skor Psikologis Mahasiswa",
    color_discrete_sequence=["SALMON"]
)

fig_box.update_layout(
    plot_bgcolor="white",
    yaxis_title="Skor Psikologis",
    title_font=dict(size=20),
    font=dict(size=14),
    yaxis=dict(showgrid=True, gridcolor="LightGrey")
)

fig_box.show()

Berdasarkan visualisasi tersebut, dapat dilihat bahwa mayoritas skor mahasiswa berada di rentang Baik hingga Cukup Baik, dengan data yang cenderung menumpuk pada skor rendah. Nilai median berada di angka 8, yang menunjukkan bahwa setengah dari responden memiliki kondisi psikologis yang tergolong baik. Sementara itu, kuartil pertama (Q1) berada di skor 4 dan kuartil ketiga (Q3) di skor 10, sehingga sebagian besar mahasiswa memiliki skor psikologis di antara nilai tersebut. Selain itu, skor maksimum tercatat sebesar 20, sedangkan minimum sebesar 4, yang memperlihatkan adanya perbedaan kondisi psikologis antarresponden.

Beberapa titik pencilan (outlier) di atas upper fence (17) memperlihatkan bahwa ada kelompok kecil mahasiswa dengan skor jauh lebih tinggi dibandingkan lainnya, sehingga kemungkinan membutuhkan perhatian khusus. Dari gambaran ini, dapat disimpulkan bahwa kondisi psikologis mayoritas mahasiswa berada pada tingkat yang cukup stabil dan positif, meskipun tetap ada sebagian kecil yang memiliki skor tinggi.

In [212]:
# Visualisasi Distribusi Kesejahteraan Psikologis Mahasiswa Berdasarkan Skor Total dengan diagram batang
fig_skor = px.bar(
    kategori_counts_skor,
    x="Kategori",
    y="Jumlah",
    title="Distribusi Kesejahteraan Psikologis Mahasiswa Berdasarkan Skor Total",
    color="Kategori",
    color_discrete_sequence=SALMON_GRAD,
    text="Jumlah"
)

fig_skor.update_traces(textposition="outside")
fig_skor.update_layout(
    plot_bgcolor="white",
    xaxis_title="Kategori Psikologis",
    yaxis_title="Jumlah Mahasiswa",
    title_font=dict(size=20),
    font=dict(size=14),
    xaxis=dict(showgrid=True, gridcolor="LightGrey"),
    yaxis=dict(showgrid=True, gridcolor="LightGrey")

)
fig_skor.show()

Visualisasi diagram batang di atas menunjukkan distribusi kesejahteraan psikologis mahasiswa berdasarkan skor total yang telah dikelompokkan ke dalam tiga kategori. Mayoritas mahasiswa berada pada kategori **Baik** dengan jumlah 96 orang, menunjukkan bahwa sebagian besar responden memiliki kondisi psikologis yang relatif stabil dan positif.

Sementara itu, terdapat 53 mahasiswa yang berada pada kategori Cukup Baik, menandakan bahwa meskipun kondisi psikologis mereka cukup baik, masih ada ruang untuk peningkatan. Hanya 4 mahasiswa yang termasuk dalam kategori Buruk, yang berarti proporsi mahasiswa dengan kesejahteraan psikologis rendah sangat kecil dibandingkan kelompok lainnya.

## Distribusi Kebutuhan Bantuan Psikologis

In [213]:
# Jumlah responden berdasarkan kebutuhan dukungan emosional & psikologis
dukungan_counts = df['kebutuhan_akan_dukungan_emosional_dan_bantuan_psikologis'].value_counts().reset_index()
dukungan_counts.columns = ['Kategori', 'Jumlah']

# Membuatdiagram lingkaran dengan Plotly Express
fig = px.pie(
    dukungan_counts,
    names='Kategori',
    values='Jumlah',
    color='Kategori',
    color_discrete_sequence=['SKYBLUE', 'SALMON'],
    title='Proporsi Kebutuhan Dukungan Emosional dan Bantuan Psikologis'
)

fig.update_traces(textposition='inside', textinfo='percent+label', rotation=90)
fig.show()

Diagram lingkaran menunjukkan proporsi kebutuhan dukungan emosional dan bantuan psikologis pada mahasiswa. Sebagian besar responden, yaitu 81 persen, menyatakan tidak membutuhkan dukungan tambahan. Sementara itu, 19 persen responden mengaku masih memerlukan dukungan emosional dan bantuan psikologis. Hasil ini menunjukkan bahwa meskipun mayoritas mahasiswa merasa cukup mampu mengelola kondisi psikologisnya secara mandiri, masih ada sebagian yang membutuhkan perhatian khusus untuk menjaga kesejahteraan mental mereka.


## Korelasi Antar Variabel

In [214]:
# Memilih variabel numerik yang penting
kolom_numerik = [
    "kemampuan_mengelola_keuangan",
    "pengaruh_fomo_terhadap_emosi",
    "frekuensi_stres_karena_finansial",
    "frekuensi_hilang_semangat_kuliah_karena_tekanan_finansial",
    "frekuensi_stres_fomo",
    "skor_psikologis"
]

# Hitung korelasi
corr_matrix = df[kolom_numerik].corr()

# Visualisasi heatmap
fig_corr = px.imshow(
    corr_matrix,
    text_auto=".2f",
    color_continuous_scale=SALMON_GRAD,
    labels=dict(x="Variabel", y="Variabel", color="Koefisien Korelasi"),
    title= "Korelasi Antar Variabel Psikologis dan Keuangan"
)

fig_corr.update_layout(
    width=1000,
    height=600,
    xaxis_title="Variabel",
    yaxis_title="Variabel",
    font=dict(size=10),
    coloraxis_colorbar=dict(title="Koefisien Korelasi", titlefont_size=16)
)

fig_corr.update_traces(textfont=dict(size=14))
fig_corr.show()


Visualisasi heatmap menunjukkan hubungan antara variabel psikologis dan keuangan dengan tingkat korelasi yang bervariasi dari -1 hingga 1, di mana warna merah yang lebih gelap menandakan hubungan yang semakin kuat. Hasil analisis menunjukkan bahwa **skor psikologis memiliki korelasi positif yang tinggi** dengan beberapa faktor seperti frekuensi stres karena finansial, frekuensi hilang semangat kuliah akibat tekanan keuangan, dan frekuensi stres akibat FOMO. Kondisi ini memperlihatkan adanya keterkaitan yang erat antara tekanan finansial dan aspek psikologis yang dialami mahasiswa. Di sisi lain, **kemampuan mengelola keuangan memiliki korelasi negatif** terhadap variabel-variabel tersebut, yang menunjukkan bahwa pengelolaan keuangan yang lebih baik berkaitan dengan tingkat tekanan psikologis yang lebih rendah. Korelasi paling kuat ditemukan antara frekuensi stres karena finansial dan kehilangan semangat kuliah akibat tekanan keuangan, menegaskan bahwa tekanan ekonomi memiliki peran penting dalam memengaruhi kesejahteraan mental mahasiswa.


## Hubungan FOMO dengan Kemampuan Mengelola Keuangan

In [215]:
# Mapping kategori FOMO
kategori_fomo = {
    1: "Tidak Pernah",
    2: "Jarang",
    3: "Kadang-kadang",
    4: "Sering",
    5: "Sangat Sering"
}
df["kategori_fomo"] = df["frekuensi_fomo_pengeluaran"].map(kategori_fomo)

# Mapping kategori kemampuan keuangan
mapping_keuangan = {
    1: "Buruk",
    2: "Kurang Baik",
    3: "Cukup Baik",
    4: "Baik",
    5: "Sangat Baik"
}
df["kategori_keuangan"] = df["kemampuan_mengelola_keuangan"].map(mapping_keuangan)

Kode tersebut berfungsi untuk mengubah data numerik menjadi kategori yang lebih mudah dipahami. Pertama, dilakukan pemetaan pada variabel frekuensi FOMO (Fear of Missing Out) terkait pengeluaran menjadi lima kategori, yaitu “Tidak Pernah”, “Jarang”, “Kadang-kadang”, “Sering”, dan “Sangat Sering”. Selanjutnya, variabel kemampuan mengelola keuangan juga dipetakan ke dalam lima kategori, yaitu “Buruk”, “Kurang Baik”, “Cukup Baik”, “Baik”, dan “Sangat Baik”. Langkah ini bertujuan agar data lebih informatif dan dapat dianalisis secara kualitatif, sehingga memudahkan dalam interpretasi hasil penelitian terkait perilaku FOMO dan kemampuan keuangan responden.


In [216]:
# Crosstab hubungan FOMO dan Kemampuan Mengelola Keuangan
crosstab = pd.crosstab(df["kategori_fomo"], df["kategori_keuangan"])

# Membuat Heatmap Hubungan FOMO dan Kemampuan Mengelola Keuangan
fig1 = px.imshow(
    crosstab,
    text_auto=True,
    color_continuous_scale=SALMON_GRAD,
    title="Heatmap Hubungan FOMO dan Kemampuan Mengelola Keuangan"
)
fig1.update_layout(xaxis_title="Kemampuan Mengelola Keuangan", yaxis_title="Tingkat FOMO", plot_bgcolor="white")
fig1.show()

# Membuat Stacked diagram batang
fomo_keuangan = (
    df.groupby(["kategori_fomo", "kategori_keuangan"])
    .size()
    .reset_index(name="Jumlah")
)

fig2 = px.bar(
    fomo_keuangan,
    x="kategori_fomo",
    y="Jumlah",
    color="kategori_keuangan",
    category_orders={
        "kategori_fomo": ["Tidak Pernah", "Jarang", "Kadang-kadang", "Sering", "Sangat Sering"],
        "kategori_keuangan": ["Sangat Baik", "Baik", "Cukup Baik", "Kurang Baik", "Buruk"]
    },
    barmode="stack",
    color_discrete_sequence=SALMON_GRAD,
    title="Distribusi Kemampuan Mengelola Keuangan Berdasarkan Tingkat FOMO",
)

fig2.update_layout(
    xaxis_title="Tingkat FOMO",
    yaxis_title="Jumlah Responden",
    plot_bgcolor="white",
    xaxis=dict(showgrid=True, gridcolor="LightGrey"),
    yaxis=dict(showgrid=True, gridcolor="LightGrey")
)
fig2.show()

Hubungan FOMO dengan kemampuan mengelola keuangan pada mahasiswa divisualisasikan melalui dua jenis grafik, yaitu heatmap dan diagram batang bertumpuk. Dalam visualisasi ini, setiap tingkat FOMO dari Tidak Pernah, Jarang, Kadang-kadang, Sering, hingga Sangat Sering ditampilkan sebagai batang yang terbagi ke dalam kategori kemampuan mengelola keuangan, mulai dari Buruk hingga Sangat Baik. Sebagai konteks dalam membaca hasil, penting dipahami bahwa semakin tinggi tingkat FOMO menandakan frekuensi FOMO yang lebih sering, sedangkan nilai yang lebih tinggi pada kemampuan mengelola keuangan menunjukkan kondisi yang lebih baik.

Melalui heatmap, distribusi responden pada setiap kombinasi tingkat FOMO dan kategori kemampuan mengelola keuangan ditampilkan melalui gradasi warna, di mana warna yang semakin mencolok menandakan jumlah responden yang lebih banyak. Visualisasi ini memudahkan untuk melihat kecenderungan pola distribusi, seperti kelompok dengan tingkat FOMO tertentu yang lebih dominan berada pada kategori kemampuan keuangan tertentu.

Diagram batang bertumpuk memperlihatkan pola hubungan yang konsisten, di mana semakin tinggi tingkat FOMO, semakin sedikit proporsi responden dengan kemampuan mengelola keuangan baik atau sangat baik. Sebaliknya, pada tingkat FOMO yang tinggi, proporsi responden dengan kemampuan keuangan kurang baik atau buruk semakin besar. Hasil ini menunjukkan kecenderungan bahwa mahasiswa yang lebih sering mengalami FOMO memiliki kemampuan mengelola keuangan yang lebih rendah dibandingkan dengan mereka yang jarang atau tidak pernah mengalaminya.


## Hubungan FOMO dengan Kesejahteraan Psikologis

In [217]:
# Crosstab hubungan FOMO dan Kesejahteraan Psikologis
crosstab = pd.crosstab(df["kategori_fomo"], df["kategori_psikologis_skor"])

# Membuat Heatmap Hubungan FOMO dan Kesejahteraan Psikologis
fig3 = px.imshow(
    crosstab,
    text_auto=True,
    color_continuous_scale=SALMON_GRAD,
    title="Heatmap Hubungan FOMO vs Kesejahteraan Psikologis"
)
fig3.update_layout(
    xaxis_title="Kesejahteraan Psikologis",
    yaxis_title="Tingkat FOMO",
    plot_bgcolor="white"
)
fig3.show()

# Membuat Stacked diagram batang
fomo_psikologis = (
    df.groupby(["kategori_fomo", "kategori_psikologis_skor"])
    .size()
    .reset_index(name="Jumlah")
)

fig4 = px.bar(
    fomo_psikologis,
    x="kategori_fomo",
    y="Jumlah",
    color="kategori_psikologis_skor",
    category_orders={
        "kategori_fomo": ["Tidak Pernah", "Jarang", "Kadang-kadang", "Sering", "Sangat Sering"],
        "kategori_psikologis_skor": ["Baik", "Cukup Baik", "Buruk"]
    },
    barmode="stack",
    color_discrete_sequence=SALMON_GRAD,
    title="Distribusi Kesejahteraan Psikologis Berdasarkan Tingkat FOMO",
)

fig4.update_layout(
    xaxis_title="Tingkat FOMO",
    yaxis_title="Jumlah Responden",
    plot_bgcolor="white",
    xaxis=dict(showgrid=True, gridcolor="LightGrey"),
    yaxis=dict(showgrid=True, gridcolor="LightGrey")
)
fig4.show()

Visualisasi hubungan antara tingkat FOMO dan kesejahteraan psikologis mahasiswa memperlihatkan pola yang konsisten melalui heatmap dan diagram batang bertumpuk. Distribusi responden menunjukkan bahwa mahasiswa dengan tingkat FOMO rendah, seperti tidak pernah dan jarang, cenderung memiliki kesejahteraan psikologis yang tergolong baik. Namun, seiring meningkatnya frekuensi FOMO ke kategori sering dan sangat sering, kondisi psikologis mulai mengalami penurunan, dengan semakin banyak responden yang berada pada kategori cukup baik bahkan buruk. Perubahan ini menunjukkan adanya keterkaitan antara intensitas FOMO dan kualitas kesehatan mental mahasiswa.

Pola distribusi tersebut memberikan gambaran penting mengenai dampak psikologis dari fenomena FOMO. Ketika rasa takut tertinggal dalam tren sosial atau pengalaman semakin sering muncul, kesejahteraan psikologis cenderung menurun dan berpotensi memengaruhi stabilitas mental secara keseluruhan. Sebaliknya, mahasiswa yang jarang mengalami FOMO umumnya memiliki kondisi psikologis yang lebih stabil dan positif. Hal ini menegaskan pentingnya kesadaran diri dan kemampuan mengelola emosi untuk mencegah dampak negatif FOMO terhadap kesehatan mental.

## Hubungan Kemampuan Mengelola Keuangan dengan Kesejahteraan Psikologis

In [218]:
# Crosstab Hubungan Kemampuan Keuangan dan Kondisi Psikologis
crosstab_keu_psiko = pd.crosstab(df["kategori_keuangan"], df["kategori_psikologis_skor"])

# Membuat Heatmap Hubungan Kemampuan Keuangan dan Kondisi Psikologis
fig5 = px.imshow(
    crosstab_keu_psiko,
    text_auto=True,
    color_continuous_scale=SALMON_GRAD,
    title="Heatmap Hubungan Kemampuan Mengelola Keuangan vs Kesejahteraan Psikologis"
)
fig5.update_layout(
    xaxis_title="Kesejahteraan Psikologis",
    yaxis_title="Kemampuan Mengelola Keuangan",
    plot_bgcolor="white"
)
fig5.show()

# Stacked diagram batang
keu_psiko = (
    df.groupby(["kategori_keuangan", "kategori_psikologis_skor"])
    .size()
    .reset_index(name="Jumlah")
)

fig6 = px.bar(
    keu_psiko,
    x="kategori_keuangan",
    y="Jumlah",
    color="kategori_psikologis_skor",
    category_orders={
        "kategori_keuangan": ["Buruk", "Kurang Baik", "Cukup Baik", "Baik", "Sangat Baik"],
        "kategori_psikologis_skor": ["Baik", "Cukup Baik", "Buruk"]
    },
    barmode="stack",
    color_discrete_sequence=SALMON_GRAD,
    title="Distribusi Kesejahteraan Psikologis Berdasarkan Kemampuan Mengelola Keuangan"
)

fig6.update_layout(
    xaxis_title="Kemampuan Mengelola Keuangan",
    yaxis_title="Jumlah Responden",
    plot_bgcolor="white",
    xaxis=dict(showgrid=True, gridcolor="LightGrey"),
    yaxis=dict(showgrid=True, gridcolor="LightGrey")
)
fig6.show()

Visualisasi hubungan antara kemampuan mengelola keuangan dan kesejahteraan psikologis mahasiswa memberikan gambaran menarik karena makna kedua variabel ini bergerak ke arah yang berlawanan. Nilai yang lebih tinggi pada kemampuan keuangan menunjukkan kondisi yang semakin baik, sedangkan skor psikologis yang lebih tinggi justru mencerminkan kondisi yang semakin buruk. Oleh karena itu, pola hubungan yang terbentuk perlu dipahami secara hati-hati.

Hasil visualisasi menunjukkan bahwa responden dengan kemampuan keuangan rendah cenderung memiliki kondisi psikologis yang lebih rentan, sedangkan mereka yang memiliki kemampuan “baik” hingga “sangat baik” umumnya berada pada kondisi psikologis yang lebih stabil. Semakin baik seseorang dalam mengelola keuangannya, semakin kecil kemungkinan mereka mengalami tekanan psikologis yang berat. Hal ini menunjukkan bahwa keterampilan finansial tidak hanya berdampak pada kestabilan ekonomi, tetapi juga berkontribusi penting terhadap kesejahteraan psikologis mahasiswa secara menyeluruh.
